**Recuerde no agregar o quitar celdas en este notebook, ni modificar su tipo. Si lo hace, el sistema automaticamente lo calificará con cero punto cero (0.0)**

Use el siguiente conjunto de datos para resolver los problemas propuestos. Use [gradetool](gradetool.md) para verificar las respuestas del notebook.

In [1]:
%%writefile data.csv
1,Vivian,Hamilton,1971-07-08,green,1
2,Karen,Holcomb,1974-05-23,green,4
3,Cody,Garrett,1973-04-22,orange,1
4,Roth,Fry,1975-01-29,black,1
5,Zoe,Conway,1974-07-03,blue,2
6,Gretchen,Kinney,1974-10-18,viole,1
7,Driscoll,Klein,1970-10-05,blue,5
8,Karyn,Diaz,1969-02-24,red,1
9,Merritt,Guy,1974-10-17,indigo,4
10,Kylan,Sexton,1975-02-28,black,4
11,Jordan,Estes,1969-12-07,indigo,4
12,Hope,Coffey,1973-12-24,green,5
13,Vivian,Crane,1970-08-27,gray,5
14,Clio,Noel,1972-12-12,red,5
15,Hope,Silva,1970-07-01,blue,5
16,Ayanna,Jarvis,1974-02-11,orange,5
17,Chanda,Boyer,1973-04-01,green,4
18,Chadwick,Knight,1973-04-29,yellow,1

Overwriting data.csv


In [2]:
%load_ext bigdata
%pig_init

In [3]:
%%pig
u = LOAD 'data.csv' USING PigStorage(',') 
    AS (id:int, 
        firstname:CHARARRAY, 
        surname:CHARARRAY, 
        birthday:CHARARRAY, 
        color:CHARARRAY, 
        quantity:INT);

## Pregunta 1

Escriba una consulta en Pig que genere la siguiente salida:

    (Vivian@Hamilton)
    (Karen@Holcomb)
    (Cody@Garrett)
    (Roth@Fry)
    (Zoe@Conway)
    (Gretchen@Kinney)
    (Driscoll@Klein)
    (Karyn@Diaz)
    (Merritt@Guy)
    (Kylan@Sexton)
    (Jordan@Estes)
    (Hope@Coffey)
    (Vivian@Crane)
    (Clio@Noel)
    (Hope@Silva)
    (Ayanna@Jarvis)
    (Chanda@Boyer)
    (Chadwick@Knight)

In [6]:
%%pig
a = FOREACH u GENERATE CONCAT(firstname,'@',surname);
dump a

(Vivian@Hamilton)
(Karen@Holcomb)
(Cody@Garrett)
(Roth@Fry)
(Zoe@Conway)
(Gretchen@Kinney)
(Driscoll@Klein)
(Karyn@Diaz)
(Merritt@Guy)
(Kylan@Sexton)
(Jordan@Estes)
(Hope@Coffey)
(Vivian@Crane)
(Clio@Noel)
(Hope@Silva)
(Ayanna@Jarvis)
(Chanda@Boyer)
(Chadwick@Knight)


## Pregunta 2

Genere una relación con el apellido y su longitud. Ordene por longitud y por apellido. Obtenga la siguiente salida.

    (Hamilton,8)
    (Garrett,7)
    (Holcomb,7)
    (Coffey,6)
    (Conway,6)

In [7]:
%%pig
b = FOREACH u GENERATE surname,SIZE(surname) as tama;
y = ORDER b BY tama DESC, surname;
y = limit y 5;
dump y;

(Hamilton,8)
(Garrett,7)
(Holcomb,7)
(Coffey,6)
(Conway,6)


## Pregunta 3

Escriba el código que genere la siguiente salida.

    (Hamilton,HAMILTON,hamilton)
    (Holcomb,HOLCOMB,holcomb)
    (Garrett,GARRETT,garrett)
    (Fry,FRY,fry)
    (Conway,CONWAY,conway)

In [8]:
%%pig
c = FOREACH u GENERATE surname, UPPER(surname),LOWER(surname);
c = limit c 5;
dump c;

(Hamilton,HAMILTON,hamilton)
(Holcomb,HOLCOMB,holcomb)
(Garrett,GARRETT,garrett)
(Fry,FRY,fry)
(Conway,CONWAY,conway)


## Pregunta 4

Obtenga los apellidos que empiecen por las letras entre la 'd' y la 'k'. La salida esperada es la siguiente:

    (Hamilton)
    (Holcomb)
    (Garrett)
    (Fry)
    (Kinney)
    (Klein)
    (Diaz)
    (Guy)
    (Estes)
    (Jarvis)
    (Knight)

In [15]:
%%pig
d = FILTER u BY (LOWER(SUBSTRING(surname,0,1)) >= 'd')
AND (LOWER(SUBSTRING(surname,0,1)) <= 'k');
f = FOREACH d GENERATE surname;
dump f;

(Hamilton)
(Holcomb)
(Garrett)
(Fry)
(Kinney)
(Klein)
(Diaz)
(Guy)
(Estes)
(Jarvis)
(Knight)


## Pregunta 5

Escriba el código equivalente a la siguiente consulta en SQL.

    SELECT
        color
    FROM 
        u 
    WHERE 
        color 
    LIKE 'b%';

In [10]:
%%pig

--f = FILTER u BY (LOWER(SUBSTRING(color,0,1)) == 'b');
--g = FOREACH f GENERATE color;
--dump g;

lluvia = FILTER u BY REGEX_EXTRACT(color, '^b', 0) == 'b';
lluvia = FOREACH lluvia GENERATE color;
dump lluvia;

(black)
(blue)
(blue)
(black)
(blue)


## Pregunta 6

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        color 
    FROM 
        u 
    WHERE 
        color NOT LIKE 'b%';

In [11]:
%%pig
--f = FILTER u BY (LOWER(SUBSTRING(color,0,1)) != 'b');
--g = FOREACH f GENERATE color;
--dump g;

lluvia = FILTER u BY REGEX_EXTRACT(color, '^[^b]', 0) != 'b';
lluvia = FOREACH lluvia GENERATE color;
dump lluvia;

(green)
(green)
(orange)
(viole)
(red)
(indigo)
(indigo)
(green)
(gray)
(red)
(orange)
(green)
(yellow)


## Pregunta 7

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname,
        color
    FROM 
        u 
    WHERE color = 'blue' AND firstname LIKE 'Z%';

In [14]:
%%pig
--h = FILTER u BY (color MATCHES 'blue') 
--AND (SUBSTRING(firstname,0,1) == 'Z');
--i = FOREACH h GENERATE firstname,color;
--dump i;

h = FILTER u BY (color MATCHES 'blue');
i = FILTER h by (REGEX_EXTRACT(firstname, '^Z', 0) == 'Z');
j = FOREACH i GENERATE firstname,color;
dump j;

(Zoe,blue)


## Pregunta 8

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT
        firstname,
        color
    FROM 
        u
    WHERE color = 'blue' OR firstname LIKE 'K%';

In [13]:
%%pig
--el operador OR me está imprimiendo una linea no pone problema pues el 
--resultado es correcto, pero la calificación la tacha de incorrecta
--j = FILTER u BY color MATCHES 'blue';
--k = FILTER u BY (REGEX_EXTRACT(firstname,'^K',0) == 'K');

j = FILTER u BY (color MATCHES 'blue') OR 
(REGEX_EXTRACT(firstname,'^K',0) == 'K');
k = FOREACH j GENERATE firstname,color;
dump k;

(Karen,green)
(Zoe,blue)
(Driscoll,blue)
(Karyn,red)
(Kylan,black)
(Hope,blue)


## Pregunta 9

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname, color 
    FROM 
        u
    WHERE color IN ('blue','black');

In [16]:
%%pig
l = FILTER u BY (color MATCHES 'blue') OR (color MATCHES 'black');
m = FOREACH l GENERATE firstname,color;
dump m;

(Roth,black)
(Zoe,blue)
(Driscoll,blue)
(Kylan,black)
(Hope,blue)


## Pregunta 10

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname, 
        color 
    FROM 
        u 
    WHERE color NOT IN ('blue','black');

In [17]:
%%pig
n = FILTER u BY NOT((color MATCHES 'blue') OR (color MATCHES 'black'));
o = FOREACH n GENERATE firstname,color;
dump o;

(Vivian,green)
(Karen,green)
(Cody,orange)
(Gretchen,viole)
(Karyn,red)
(Merritt,indigo)
(Jordan,indigo)
(Hope,green)
(Vivian,gray)
(Clio,red)
(Ayanna,orange)
(Chanda,green)
(Chadwick,yellow)


## Pregunta 11

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT
        firstname,
        color
    FROM 
        u 
    WHERE 
        color REGEXP '^b';

In [18]:
%%pig
p= FILTER u BY (REGEX_EXTRACT(color,'^b',0) == 'b');
q= FOREACH p GENERATE firstname,color;
dump q;

(Roth,black)
(Zoe,blue)
(Driscoll,blue)
(Kylan,black)
(Hope,blue)


## Pregunta 12

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT
        firstname,
        color
    FROM 
        u 
    WHERE 
        color REGEXP '^[^b]';

In [19]:
%%pig
p= FILTER u BY (REGEX_EXTRACT(color,'^[^b]',0) != 'b');
q= FOREACH p GENERATE firstname,color;
dump q;

(Vivian,green)
(Karen,green)
(Cody,orange)
(Gretchen,viole)
(Karyn,red)
(Merritt,indigo)
(Jordan,indigo)
(Hope,green)
(Vivian,gray)
(Clio,red)
(Ayanna,orange)
(Chanda,green)
(Chadwick,yellow)


## Pregunta 13

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname,
        color
    FROM 
        u
    WHERE 
        color REGEXP 'blue|green';

In [38]:
%%pig
p= FILTER u BY (REGEX_EXTRACT(color,'blue|green',0) == 'blue') 
OR (REGEX_EXTRACT(color,'blue|green',0) == 'green');
q= FOREACH p GENERATE firstname,color;
dump q;

(Vivian,green)
(Karen,green)
(Zoe,blue)
(Driscoll,blue)
(Hope,green)
(Hope,blue)
(Chanda,green)


## Pregunta 14

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname,
        color
    FROM 
        u
    WHERE 
        color REGEXP '.n';

In [48]:
%%pig
p= FILTER u BY (LOWER(SUBSTRING(color,((int)(SIZE(color)-1))
                                ,(int)SIZE(color)))) == 'n';
q= FOREACH p GENERATE firstname,color;
dump q;

(Vivian,green)
(Karen,green)
(Hope,green)
(Chanda,green)


## Pregunta 15

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname,
        color 
    FROM 
        u 
    WHERE 
        color REGEXP '[aeiou]$';

In [49]:
%%pig
llueve = FILTER u BY REGEX_EXTRACT(color, '[aeiou]$', 0) != '';
llueve = FOREACH llueve GENERATE firstname,color;
dump llueve;

(Cody,orange)
(Zoe,blue)
(Gretchen,viole)
(Driscoll,blue)
(Merritt,indigo)
(Jordan,indigo)
(Hope,blue)
(Ayanna,orange)


## Pregunta 16

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        REGEX_EXTRACT(birthday, '....-..-..', 2) 
    FROM 
        u;

In [25]:
%%pig
llueve = foreach u generate 
REGEX_EXTRACT(birthday, '(....)-(..)-(..)', 2);
dump llueve;

(07)
(05)
(04)
(01)
(07)
(10)
(10)
(02)
(10)
(02)
(12)
(12)
(08)
(12)
(07)
(02)
(04)
(04)


## Pregunta 17

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT  
        firstname,
        SUBSTRING_INDEX(firstname, 'a', 1)
    FROM 
        u;

In [51]:
%%pig

q= FOREACH u GENERATE INDEXOF(firstname,'a',1) as index;
--la respuesta del ejercicio debería de ser:
-- q= FOREACH u GENERATE firstname,INDEXOF(firstname,'a',1) as index;
--p = FOREACH q GENERATE 
--(CASE 
-- WHEN index == -1 THEN firstname
-- WHEN index != -1 THEN SUBSTRING(firstname,0,(int)index)
-- END);
dump q;

(4)
(1)
(-1)
(-1)
(-1)
(-1)
(-1)
(1)
(-1)
(3)
(4)
(-1)
(4)
(-1)
(-1)
(2)
(2)
(2)


## Pregunta 18

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname 
    FROM 
        u 
    WHERE 
        SUBSTRING(firstname, 0, 1) >= 'm';

In [27]:
%%pig
x = FILTER u BY SUBSTRING(LOWER(firstname),0,1) >= 'm';
x = FOREACH x GENERATE firstname;
dump x;

(Vivian)
(Roth)
(Zoe)
(Merritt)
(Vivian)


## Pregunta 19

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname, 
        LOCATE('ia', firstname) 
    FROM 
        u;

In [52]:
%%pig
--q= FOREACH u GENERATE firstname,INDEXOF(firstname,'ia',1) as index;
q= FOREACH u GENERATE INDEXOF(firstname,'ia',1) as index;
dump q;

(3)
(-1)
(-1)
(-1)
(-1)
(-1)
(-1)
(-1)
(-1)
(-1)
(-1)
(-1)
(3)
(-1)
(-1)
(-1)
(-1)
(-1)


## Pregunta 20

Escriba el código en Pig para manipulación de fechas que genere sea equivalente a la siguiente consulta SQL.

    SELECT 
        birthday, 
        DATE_FORMAT(birthday, "yyyy"),
        DATE_FORMAT(birthday, "yy"),
    FROM 
        persons
    LIMIT
        5;

In [54]:
%%pig
--respuesta equivocada
--date_data = FOREACH u generate birthday,
--          ToString(ToDate(birthday,'yyyy-MM-dd'),'yyyy'),
--            ToString(ToDate(birthday,'yyyy-MM-dd'),'yy');
--date_data = LIMIT date_data 5;
date_data = FOREACH u generate 
        ToString(ToDate(birthday,'yyyy-MM-dd'),'yyyy'),
        ToString(ToDate(birthday,'yyyy-MM-dd'),'yy');
dump date_data;

(1971,71)
(1974,74)
(1973,73)
(1975,75)
(1974,74)
(1974,74)
(1970,70)
(1969,69)
(1974,74)
(1975,75)
(1969,69)
(1973,73)
(1970,70)
(1972,72)
(1970,70)
(1974,74)
(1973,73)
(1973,73)


## Pregunta 21

Escriba el código en Pig para manipulación de fechas que genere la siguiente salida.

    (1971-07-08,jul,07,7)
    (1974-05-23,may,05,5)
    (1973-04-22,abr,04,4)
    (1975-01-29,ene,01,1)
    (1974-07-03,jul,07,7)
    (1974-10-18,oct,10,10)
    (1970-10-05,oct,10,10)
    (1969-02-24,feb,02,2)
    (1974-10-17,oct,10,10)
    (1975-02-28,feb,02,2)
    (1969-12-07,dic,12,12)
    (1973-12-24,dic,12,12)
    (1970-08-27,ago,08,8)
    (1972-12-12,dic,12,12)
    (1970-07-01,jul,07,7)
    (1974-02-11,feb,02,2)
    (1973-04-01,abr,04,4)
    (1973-04-29,abr,04,4)

In [34]:
%%pig
date_data = FOREACH u generate birthday,
ToString( ToDate(birthday,'yyyy-MM-dd'), 'MMM' ) as mes;

date_data = FOREACH date_data GENERATE birthday,(CASE     
                              WHEN mes == 'Jan' THEN 'ene' 
                              WHEN mes == 'Feb' THEN 'feb' 
                              WHEN mes == 'Mar' THEN 'mar' 
                              WHEN mes == 'May' THEN 'may' 
                              WHEN mes == 'Apr' THEN 'abr' 
                              WHEN mes == 'Jun' THEN 'jun'
                              WHEN mes == 'Jul' THEN 'jul' 
                              WHEN mes == 'Aug' THEN 'ago' 
                              WHEN mes == 'Sep' THEN 'sep' 
                              WHEN mes == 'Oct' THEN 'oct'
                              WHEN mes == 'Nov' THEN 'nov' 
                              WHEN mes == 'Dec' THEN 'dic' 
                              END),
                              (CASE
            WHEN GetMonth(ToDate(birthday,'yyyy-MM-dd')) < 10 THEN 
            CONCAT('0',(chararray)GetMonth(ToDate(birthday,'yyyy-MM-dd')))
            ELSE (chararray)GetMonth(ToDate(birthday,'yyyy-MM-dd'))
            END),
            GetMonth(ToDate(birthday,'yyyy-MM-dd'));
                              
dump date_data;

(1971-07-08,jul,07,7)
(1974-05-23,may,05,5)
(1973-04-22,abr,04,4)
(1975-01-29,ene,01,1)
(1974-07-03,jul,07,7)
(1974-10-18,oct,10,10)
(1970-10-05,oct,10,10)
(1969-02-24,feb,02,2)
(1974-10-17,oct,10,10)
(1975-02-28,feb,02,2)
(1969-12-07,dic,12,12)
(1973-12-24,dic,12,12)
(1970-08-27,ago,08,8)
(1972-12-12,dic,12,12)
(1970-07-01,jul,07,7)
(1974-02-11,feb,02,2)
(1973-04-01,abr,04,4)
(1973-04-29,abr,04,4)


## Pregunta 22

Escriba el código en Pig para manipulación de fechas que genere la siguiente salida:

    (1971-07-08,08,8,jue,jueves)
    (1974-05-23,23,23,jue,jueves)
    (1973-04-22,22,22,dom,domingo)
    (1975-01-29,29,29,mié,miércoles)
    (1974-07-03,03,3,mié,miércoles)
    (1974-10-18,18,18,vie,viernes)
    (1970-10-05,05,5,lun,lunes)
    (1969-02-24,24,24,lun,lunes)
    (1974-10-17,17,17,jue,jueves)
    (1975-02-28,28,28,vie,viernes)
    (1969-12-07,07,7,dom,domingo)
    (1973-12-24,24,24,lun,lunes)
    (1970-08-27,27,27,jue,jueves)
    (1972-12-12,12,12,mar,martes)
    (1970-07-01,01,1,mié,miércoles)
    (1974-02-11,11,11,lun,lunes)
    (1973-04-01,01,1,dom,domingo)
    (1973-04-29,29,29,dom,domingo)

In [36]:
%%pig
date_data = FOREACH u generate birthday,
            ToString( ToDate(birthday,'yyyy-MM-dd'), 'EEE' ) as dia;

date_data = FOREACH date_data GENERATE birthday,(CASE     
                              WHEN dia == 'Mon' THEN 'lunes' 
                              WHEN dia == 'Tue' THEN 'martes' 
                              WHEN dia == 'Wed' THEN 'miércoles' 
                              WHEN dia == 'Thu' THEN 'jueves' 
                              WHEN dia == 'Fri' THEN 'viernes' 
                              WHEN dia == 'Sat' THEN 'sábado'
                              WHEN dia == 'Sun' THEN 'domingo' 
                              END) as dia_semana;
date_data = FOREACH date_data GENERATE birthday,
(CASE
            WHEN GetDay(ToDate(birthday,'yyyy-MM-dd')) < 10 THEN 
            CONCAT('0',(chararray)GetDay(ToDate(birthday,'yyyy-MM-dd')))
            ELSE (chararray)GetDay(ToDate(birthday,'yyyy-MM-dd'))
            END),
                              GetDay(ToDate(birthday,'yyyy-MM-dd')),
                              SUBSTRING(dia_semana,0,3),
                              dia_semana;
                              
dump date_data;

(1971-07-08,08,8,jue,jueves)
(1974-05-23,23,23,jue,jueves)
(1973-04-22,22,22,dom,domingo)
(1975-01-29,29,29,mié,miércoles)
(1974-07-03,03,3,mié,miércoles)
(1974-10-18,18,18,vie,viernes)
(1970-10-05,05,5,lun,lunes)
(1969-02-24,24,24,lun,lunes)
(1974-10-17,17,17,jue,jueves)
(1975-02-28,28,28,vie,viernes)
(1969-12-07,07,7,dom,domingo)
(1973-12-24,24,24,lun,lunes)
(1970-08-27,27,27,jue,jueves)
(1972-12-12,12,12,mar,martes)
(1970-07-01,01,1,mié,miércoles)
(1974-02-11,11,11,lun,lunes)
(1973-04-01,01,1,dom,domingo)
(1973-04-29,29,29,dom,domingo)


## Pregunta 23

Cuente la cantidad de personas nacidas por año.

In [37]:
%%pig
x = FOREACH u GENERATE SUBSTRING(birthday,0,4) as year;
y = GROUP x BY year;
z = FOREACH y GENERATE group,COUNT(x);
dump z;

(1969,2)
(1970,3)
(1971,1)
(1972,1)
(1973,4)
(1974,5)
(1975,2)


---

In [ ]:
!rm pig_* data.csv

---